<p align="center">
<img src="https://www.uao.edu.co/wp-content/uploads/2024/12/uao-logo-2-04.webp" width=15%>


<h2>UNIVERSIDAD AUTÓNOMA DE OCCIDENTE</strong></h2>
<h3>03/06/2025 CALI - COLOMBIA</strong></h3>
<h3><strong>MAESTRIA EN INTELIGENCIA ARTIFICIAL Y CIENCIA DE DATOS</strong></h3>
<h3><strong>ETL (EXTRACT, TRANSFORM AND LOAD)</strong></h3>
<h3><strong>EJERCICIO EN CLASE 2 </strong> TRANSFORMACIONES</h3>
<h3><strong>Profesor:</strong> JAVIER ALEJANDRO VERGARA ZORRILLA</h3>
<h3><strong>Alumno:</strong>><font color='lighblue'> 22500214 Yoniliman Galvis Aguirre </font></h3>

# EJERCICIO ETL #2

## Contexto
Este conjunto de datos proviene de una empresa de comercio electrónico, que contiene diversa información sobre los productos disponibles en la tienda.

## Ejercicio
*   Realiza todas las transformaciones necesarias para obtener un conjunto de datos limpio con las características requeridas para entrenar un modelo de aprendizaje automático que prediga si un producto es nuevo o usado.

*   Una característica es una columna con información importante o relevante para resolver el problema.

*   Toma en cuenta todas las consideraciones y supuestos que necesites. En la carpeta de Google Drive puedes encontrar el archivo data_clean.csv, el cual puedes usar como ejemplo de salida.

*   Realiza todo el Análisis Exploratorio de Datos (EDA) que consideres necesario, utiliza gráficos como apoyo y aplica todas las transformaciones requeridas.

# Verificar Kernel
Verificamos si el ambiente jupyter esta ejecutando el kernel en el entorno correcto, el resultado de las dos rutas debe coincidir, de lo contrario se debe de cambiar el kernel del jupyter notebook, una opcion es correr el enviroment desde poetry, en la terminal ejecute:
```bash
poetry run jupyter notebook
```
esto abrirá una version web de jupyter, en otro caso cambie el kernel y use los venv disponibles

Si el notebook esta ejecutando un kernel diferente a la carpeta del proyecto cuando instale librerías se presentarán fallas en la ejecucion del código del notebook

In [1]:
import subprocess
import shutil

# Ejecutar el comando de poetry desde Python
result = subprocess.run(['poetry', 'env', 'info', '--path'], capture_output=True, text=True)

# Obtener la ruta del entorno virtual
env_path = result.stdout.strip()

# Obtener la ruta del ejecutable de Python activo
python_path = shutil.which("python")

print(f"El entorno virtual activo de poetry está en: {env_path}")
print(f"El entorno virtual activo del kernel en el notebook está en: {python_path}")

El entorno virtual activo de poetry está en: /home/ygalvis/Documents/Study/ETL_Ejercicio2/.venv
El entorno virtual activo del kernel en el notebook está en: /home/ygalvis/Documents/Study/ETL_Ejercicio2/.venv/bin/python


# Cargar librerías

In [2]:
import pandas as pd
import jsonlines
from tqdm import tqdm
import numpy as np

# Cargar Datos desde archivo JSONL

In [3]:
# Esta funcion va a Leer el archivo JSONL usando la librería jsonlines

# 🔹 Función para verificar si un valor es vacío
def es_vacio(x):
    """ Verifica si x es un diccionario vacío, lista vacía, array vacío o NaN """
    return (
        (isinstance(x, dict) and len(x) == 0) or 
        (isinstance(x, list) and len(x) == 0) or 
        (isinstance(x, np.ndarray) and x.size == 0) or 
        pd.isna(x)
    )

# 🔹 Función para leer el JSONL y limpiar el DataFrame
def leer_jsonl(ruta_archivo, ruta_exportar):
    """
    *   Leer un archivo JSONL y cargarlo en un DataFrame.
    *   Eliminar columnas con valores nulos, listas vacías ([]), o diccionarios vacíos ({}).
    *   Exportar los encabezados y la primera fila a un archivo de texto.

    Args:
        ruta_archivo (str): Ruta del archivo JSONL.
        ruta_exportar (str): Ruta del archivo de texto donde se exportarán los encabezados y la primera fila.

    Returns:
        pd.DataFrame: Un DataFrame limpio con las columnas vacías eliminadas.
    """
    data = []
    try:
        with jsonlines.open(ruta_archivo) as reader:
            for obj in tqdm(reader, desc="Leyendo líneas del archivo JSONL"):
                data.append(obj)
    except FileNotFoundError:
        print("❌ Error: No se encontró el archivo. Verifica la ruta.")
        return pd.DataFrame()
    except jsonlines.InvalidLineError as e:
        print(f"❌ Error al leer una línea del archivo. Revisa el JSON: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        return pd.DataFrame()

    # Convertir a DataFrame
    df = pd.DataFrame(data)

    if df.empty:
        print("⚠️ El archivo JSONL está vacío o no se pudo cargar correctamente.")
        return df

    # 🔹 Identificar columnas a eliminar
    columnas_a_eliminar = []
    for col in df.columns:
        try:
            # Si hay listas, aplanamos con explode() para evaluar correctamente
            df_temp = df[col].explode() if df[col].apply(lambda x: isinstance(x, list)).any() else df[col]

            # Si todos los valores en la columna son vacíos según `es_vacio`
            if df_temp.apply(es_vacio).all():
                columnas_a_eliminar.append(col)
        except Exception as e:
            print(f"⚠️ Error al procesar la columna {col}: {e}")

    # 🔹 Eliminar las columnas vacías
    df.drop(columns=columnas_a_eliminar, inplace=True)
    print("🗑 Columnas eliminadas:", columnas_a_eliminar)

    # 🔹 Exportar encabezados y primera fila alineados
    with open(ruta_exportar, "w", encoding="utf-8") as f:
        columnas = df.columns.tolist()
        valores = df.iloc[0].astype(str).tolist() if not df.empty else [""] * len(columnas)
        max_len = max(len(col) for col in columnas)

        f.write("Encabezado y primer fila:\n\n")
        for col, val in zip(columnas, valores):
            f.write(f"{col.rjust(max_len)} : {val}\n")

    print(f"✅ Archivo exportado a: {ruta_exportar}")
    return df

In [4]:
ruta_archivo = 'Dataset/MLA_100k.jsonlines'
ruta_exportar = 'Dataset/header_firstrow.txt'
df = leer_jsonl(ruta_archivo, ruta_exportar)

# Mostrar todas las columnas en formato vertical con las dos primeras filas
pd.options.display.max_rows = None
print(df.head(2).T)

Leyendo líneas del archivo JSONL: 100000it [00:04, 20410.66it/s]


🗑 Columnas eliminadas: ['coverage_areas', 'differential_pricing', 'subtitle']
✅ Archivo exportado a: Dataset/header_firstrow.txt
                                                                                  0  \
seller_address                    {'comment': '', 'longitude': -58.3986709, 'id'...   
warranty                                                                       None   
sub_status                                                                       []   
condition                                                                       new   
seller_contact                                                                 None   
deal_ids                                                                         []   
base_price                                                                     80.0   
shipping                          {'local_pick_up': True, 'methods': [], 'tags':...   
non_mercado_pago_payment_methods  [{'description': 'Transferencia bancaria', 'id...   
s

In [5]:
# 🔹 Identificar columnas con diccionarios o listas (incluyendo vacíos)
columnas_anidadas = []

for col in df.columns:
    # Ignorar columnas completamente vacías
    if df[col].dropna().empty:
        continue
    
    # 🔹 Verificar si la columna contiene diccionarios o listas
    if df[col].apply(lambda x: isinstance(x, (dict, list))).any():
        columnas_anidadas.append(col)

print("Columnas con estructuras anidadas:", columnas_anidadas)
df_anidadas = df[columnas_anidadas]

Columnas con estructuras anidadas: ['seller_address', 'sub_status', 'seller_contact', 'deal_ids', 'shipping', 'non_mercado_pago_payment_methods', 'variations', 'location', 'attributes', 'tags', 'descriptions', 'pictures', 'geolocation']


In [7]:
df_anidadas.head()

,seller_address,sub_status,seller_contact,deal_ids,shipping,non_mercado_pago_payment_methods,variations,location,attributes,tags,descriptions,pictures,geolocation
0,"{'comment': '', 'longitude': -58.3986709, 'id'...",[],None,[],"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",[],{},[],[dragged_bids_and_visits],[{'id': 'MLA578052519-912855983'}],"[{'size': '500x375', 'secure_url': 'https://a2...","{'latitude': -34.6280698, 'longitude': -58.398..."
1,"{'comment': '', 'longitude': -58.5059173, 'id'...",[],None,[],"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",[],{},[],[],[{'id': 'MLA581565358-930764806'}],"[{'size': '499x334', 'secure_url': 'https://a2...","{'latitude': -34.5935524, 'longitude': -58.505..."
2,"{'comment': '', 'longitude': -58.4143948, 'id'...",[],None,[],"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",[],{},[],[dragged_bids_and_visits],[{'id': 'MLA578780872-916478256'}],"[{'size': '375x500', 'secure_url': 'https://a2...","{'latitude': -34.6233907, 'longitude': -58.414..."
3,"{'comment': '', 'longitude': -58.4929208, 'id'...",[],None,[],"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",[],{},[],[],[{'id': 'MLA581877385-932309698'}],"[{'size': '441x423', 'secure_url': 'https://a2...","{'latitude': -34.6281894, 'longitude': -58.492..."
4,"{'comment': '', 'longitude': -58.5495042, 'id'...",[],None,[],"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",[],{},[],[dragged_bids_and_visits],[{'id': 'MLA576112692-902981678'}],"[{'size': '375x500', 'secure_url': 'https://a2...","{'latitude': -34.6346547, 'longitude': -58.549..."


In [34]:
df_expanded_list = []  # Lista para almacenar DataFrames expandidos

# Expandir cada columna anidada
def leer_jsonl_y_limpiar_columnas(ruta_archivo, ruta_exportar):
    """
    Lee un archivo JSONL, elimina las columnas vacías y duplicadas, expande columnas anidadas,
    y exporta los encabezados y la primera fila a un archivo de texto.

    Args:
        ruta_archivo (str): La ruta al archivo JSONL.
        ruta_exportar (str): La ruta al archivo de texto donde se exportarán los encabezados y la primera fila.

    Returns:
        pd.DataFrame: Un DataFrame de pandas con las columnas vacías, duplicadas eliminadas y columnas anidadas expandidas.
    """
    data = []
    try:
        with jsonlines.open(ruta_archivo) as reader:
            for obj in tqdm(reader, desc="Leyendo líneas del archivo JSONL"):
                data.append(obj)
    except FileNotFoundError:
        print("El archivo no se encontró. Verifica la ruta.")
        return pd.DataFrame()  # Retorna un DataFrame vacío en caso de error
    except jsonlines.InvalidLineError as e:
        print(f"Error al leer una línea del archivo: {e}")
        return pd.DataFrame()  # Retorna un DataFrame vacío en caso de error

    # Convertir la lista de diccionarios a un DataFrame de pandas
    df = pd.DataFrame(data)


    # Eliminar columnas que contienen solo valores vacíos
    df = df.dropna(axis=1, how='all')
    #df = df[[col for col in df if not all(es_vacio(i) for i in df[col])]]

    # Eliminar columnas duplicadas
    #df = df.T.drop_duplicates().T

    # Identificar columnas anidadas
    columnas_anidadas = [col for col in df.columns if isinstance(df[col].iloc[0], dict)]

    # Expandir cada columna anidada
    df_expanded_list = []
    for col in columnas_anidadas:
        if col in df.columns:
            df_expanded = pd.json_normalize(df[col], sep="_")
            
            # Renombrar la columna 'id' si existe en df_expanded para evitar conflictos
            if "id" in df_expanded.columns:
                df_expanded.rename(columns={"id": f"{col}_id"}, inplace=True)

            df_expanded_list.append(df_expanded)

    # Concatenar los DataFrames expandidos
    if df_expanded_list:
        df_expanded_final = pd.concat(df_expanded_list, axis=1)
        # Unir con el DataFrame original sin eliminar la columna 'id' original
        df = df.drop(columns=columnas_anidadas).join(df_expanded_final)

    # Exportar los encabezados de las columnas y la primera fila a un archivo de texto
    with open(ruta_exportar, "w") as f:
        columnas = df.columns.tolist()
        valores = df.iloc[0].astype(str).tolist()
        
        max_len = max(len(col) for col in columnas)  # Longitud máxima de columna

        for col, val in zip(columnas, valores):
            f.write("{:>{width}} : {}\n".format(col, val, width=max_len))  # Alinear nombres

    return df

# Uso de la función
ruta_archivo = 'Dataset/MLA_100k.jsonlines'
ruta_exportar = 'Dataset/header_firstrow.txt'
df_final = leer_jsonl_y_limpiar_columnas(ruta_archivo, ruta_exportar)

# Mostrar la primera fila transpuesta
print(df_final.head(1).T)


Leyendo líneas del archivo JSONL: 100000it [00:05, 17259.07it/s]


ValueError: columns overlap but no suffix specified: Index(['tags'], dtype='object')

In [28]:
# Eliminar columnas que contienen solo valores vacíos
df_final = df_final.dropna(axis=1, how='all')
df_final = df_final[[col for col in df_final if not all(es_vacio(i) for i in df_final[col])]]
# Eliminar columnas duplicadas
df_final = df_final.T.drop_duplicates().T
df_final.head(1).T

TypeError: unhashable type: 'dict'